This file allows for testing the GraphWorld setup with GNN implementations.
It is currently set up to test the SSL methods for the JL benchmarker.

Through this notebook you can attach a debugger.
Note that graph_tool does not work on windows, so we cannot use the graph generators.
Instead, we use the standard datasets from PyG.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from graph_world.self_supervised_learning.benchmarker_jl import NNNodeBenchmarkerJL
from graph_world.models.basic_gnn import GCN, SuperGAT
from torch_geometric.datasets import Planetoid

from graph_world.self_supervised_learning.pretext_tasks.contrastive_based import *


In [2]:
# Parameter setup (for cora)
benchmark_params = {
    'epochs' : 50,
    'lr' : 0.01,
    'lambda' : 1
}

h_params = {
    'in_channels' : 1433,
    'hidden_channels' : 16,
    'num_layers' : 2,
    'dropout' : 0.5,
    'moving_average_decay' : 0.99, 
    'edge_mask_ratio1' : 0.2,
    'edge_mask_ratio2' : 0.4,
    'feature_mask_ratio1' : 0.4, 
    'feature_mask_ratio2' : 0.2
}

generator_config = {
    'num_clusters' : 7,
}

pretext_tasks = BGRL

In [3]:
# Get dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')[0]

In [4]:
# Training. You can attach a debugger to w/e is needed inside train
benchmarker = NNNodeBenchmarkerJL(generator_config=generator_config, model_class=GCN, 
                benchmark_params=benchmark_params, h_params=h_params, pretext_tasks=pretext_tasks)
benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=dataset.val_mask, test_mask=dataset.test_mask)
benchmarker.train(data=dataset, tuning_metric="rocauc_ovr", tuning_metric_is_loss=False)

GCN(1433, 16, num_layers=2)


c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: User

([6.379831790924072,
  5.9493303298950195,
  5.495417594909668,
  5.201396465301514,
  4.900965690612793,
  4.587197780609131,
  4.3284196853637695,
  4.11917781829834,
  3.920740842819214,
  3.875570774078369,
  3.718332290649414,
  3.528822898864746,
  3.4606292247772217,
  3.3197202682495117,
  3.238189458847046,
  3.096330404281616,
  3.0698447227478027,
  3.112044334411621,
  2.918398380279541,
  2.8514881134033203,
  2.8076131343841553,
  2.7996137142181396,
  2.721116542816162,
  2.675870418548584,
  2.6876721382141113,
  2.5284335613250732,
  2.5734338760375977,
  2.5118021965026855,
  2.4129977226257324,
  2.291210889816284,
  2.493196725845337,
  2.250030755996704,
  2.290243148803711,
  2.2315845489501953,
  2.121192455291748,
  2.0973048210144043,
  2.0810277462005615,
  2.061685562133789,
  2.152270793914795,
  1.979538083076477,
  2.046369791030884,
  1.950014352798462,
  2.0259411334991455,
  1.9872636795043945,
  1.9226691722869873,
  1.9288034439086914,
  2.03916835784